In [24]:
import requests
import os
import time
import pandas as pd
import aiohttp
import asyncio
import json
import nest_asyncio
nest_asyncio.apply()

In [25]:
TARGET = '../Satun-Phang-Nga/final-Phang-Nga/'
OUTPUT = '../Satun-Phang-Nga/final-Phang-Nga-SunriseSunset/'

In [26]:
if not os.path.exists(OUTPUT):
    os.makedirs(OUTPUT)

In [27]:
excel_files = [f for f in os.listdir(TARGET) if f.endswith('.xlsx')]

In [28]:
print(excel_files)

['1-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx', '2-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx', '3-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx', '4-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx', '5-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx', '6-2024-02-02-Thai-Mueang-Thai-Mueang-Phang-nga.xlsx', '7-2024-02-02-Thai-Mueang-Thai-Mueang-Phang-nga.xlsx', '8-2024-02-02-Thai-Mueang-Thai-Mueang-Phang-nga.xlsx']


In [29]:
# Function to process each Excel file
async def process_file(file_path, filename):
    print(f'Loading {filename}')
    df = pd.read_excel(file_path)

    # Extract latitude, longitude, and date data for API requests
    temp = []
    for i, data in df.iterrows():
        lat = data['latitude']
        lon = data['longitude']
        date = str(data['date']).replace('/', '-')
        temp.append([lat, lon, date])

    # Asynchronous function to fetch data from the API with retry logic
    async def fetch_data(session, url, retries=3, backoff_factor=1.0):
        for attempt in range(retries):
            try:
                async with session.get(url) as response:
                    if response.status == 200:
                        return await response.json()
                    else:
                        print(f"Error response {response.status} for URL: {url}")
            except (ClientConnectorError, ClientOSError, ServerDisconnectedError) as e:
                print(f"Request failed: {e}. Retrying ({attempt + 1}/{retries})...")
                await asyncio.sleep(backoff_factor * (2 ** attempt))
        print(f"Failed to fetch data from {url} after {retries} retries.")
        return None

    # Asynchronous function to manage API requests
    async def fetch_all_data():
        urls = [f"https://api.sunrisesunset.io/json?lat={lat}&lng={lon}&date_start={date}&date_end={date}" for lat, lon, date in temp]
        responses = []
        async with aiohttp.ClientSession() as session:
            tasks = [fetch_data(session, url) for url in urls]
            responses = await asyncio.gather(*tasks)
        return [response for response in responses if response is not None]

    responses = await fetch_all_data()

    if not responses:
        print(f"No data fetched for {filename}. Skipping file.")
        return

    # Process the API responses and convert them to a dataframe
    responses = [d['results'][0] for d in responses]
    df_api = pd.DataFrame.from_dict(responses)
    df_api = df_api[['date', 'sunrise', 'sunset']]
    df_api['date'] = df_api['date'].replace('-', '/', regex=True)

    # Merge the API data with the original dataframe
    df['sunrise'] = pd.to_datetime(df_api['sunrise'])
    df['sunset'] = pd.to_datetime(df_api['sunset'])
    df['durationOfDay'] = (df['sunset'] - df['sunrise']).dt.total_seconds() / 3600

    # Convert sunrise and sunset columns to time format
    df['sunrise'] = df['sunrise'].dt.time
    df['sunset'] = df['sunset'].dt.time

    # Select and reorder the relevant columns
    df = df[['date', 'latitude', 'longitude', 'uvb', 'e', 'stl1', 'sp', 'tp', 'd2m', 'minTemp', 'meanTemp', 'maxTemp', 'RH', 'WS', 'sunrise', 'sunset', 'durationOfDay']]

    # Save the updated dataframe to a new Excel file
    output_file = os.path.join(OUTPUT, f'{filename.split(".")[0]}-durationOfDay.xlsx')
    df.to_excel(output_file, index=False)

    print(f'Saved updated dataframe to {output_file}')

# Run the processing for each file
async def main():
    tasks = [process_file(os.path.join(TARGET, filename), filename) for filename in excel_files]
    await asyncio.gather(*tasks)

asyncio.run(main())

Loading 1-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
Loading 2-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
Loading 3-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
Loading 4-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
Loading 5-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga.xlsx
Loading 6-2024-02-02-Thai-Mueang-Thai-Mueang-Phang-nga.xlsx
Loading 7-2024-02-02-Thai-Mueang-Thai-Mueang-Phang-nga.xlsx
Loading 8-2024-02-02-Thai-Mueang-Thai-Mueang-Phang-nga.xlsx


C:\Users\User\AppData\Local\Temp\ipykernel_2728\3367098157.py:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunrise'] = pd.to_datetime(df_api['sunrise'])
C:\Users\User\AppData\Local\Temp\ipykernel_2728\3367098157.py:52: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunset'] = pd.to_datetime(df_api['sunset'])
C:\Users\User\AppData\Local\Temp\ipykernel_2728\3367098157.py:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunrise'] = pd.to_datetime(df_api['sunrise'])
C:\Users\User\AppData\Local\Temp\ipykernel_2728\3367098157.py:52: UserWarning: Could not infer f

Saved updated dataframe to ../Satun-Phang-Nga/final-Phang-Nga-SunriseSunset/4-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga-durationOfDay.xlsx
Saved updated dataframe to ../Satun-Phang-Nga/final-Phang-Nga-SunriseSunset/3-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga-durationOfDay.xlsx
Saved updated dataframe to ../Satun-Phang-Nga/final-Phang-Nga-SunriseSunset/2-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga-durationOfDay.xlsx


C:\Users\User\AppData\Local\Temp\ipykernel_2728\3367098157.py:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunrise'] = pd.to_datetime(df_api['sunrise'])
C:\Users\User\AppData\Local\Temp\ipykernel_2728\3367098157.py:52: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunset'] = pd.to_datetime(df_api['sunset'])
C:\Users\User\AppData\Local\Temp\ipykernel_2728\3367098157.py:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunrise'] = pd.to_datetime(df_api['sunrise'])
C:\Users\User\AppData\Local\Temp\ipykernel_2728\3367098157.py:52: UserWarning: Could not infer f

Saved updated dataframe to ../Satun-Phang-Nga/final-Phang-Nga-SunriseSunset/7-2024-02-02-Thai-Mueang-Thai-Mueang-Phang-nga-durationOfDay.xlsx
Saved updated dataframe to ../Satun-Phang-Nga/final-Phang-Nga-SunriseSunset/6-2024-02-02-Thai-Mueang-Thai-Mueang-Phang-nga-durationOfDay.xlsx
Saved updated dataframe to ../Satun-Phang-Nga/final-Phang-Nga-SunriseSunset/5-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga-durationOfDay.xlsx
Saved updated dataframe to ../Satun-Phang-Nga/final-Phang-Nga-SunriseSunset/1-2024-02-02-Na-Toei-Thai-Mueang-Phang-nga-durationOfDay.xlsx
Saved updated dataframe to ../Satun-Phang-Nga/final-Phang-Nga-SunriseSunset/8-2024-02-02-Thai-Mueang-Thai-Mueang-Phang-nga-durationOfDay.xlsx


C:\Users\User\AppData\Local\Temp\ipykernel_2728\3367098157.py:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunrise'] = pd.to_datetime(df_api['sunrise'])
C:\Users\User\AppData\Local\Temp\ipykernel_2728\3367098157.py:52: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunset'] = pd.to_datetime(df_api['sunset'])
